In [1]:
!unzip Images.zip

Streaming output truncated to the last 5000 lines.
  inflating: Images/00000630.jpg     
  inflating: Images/00000631.jpg     
  inflating: Images/00000632.jpg     
  inflating: Images/00000633.jpg     
  inflating: Images/00000634.jpg     
  inflating: Images/00000635.jpg     
  inflating: Images/00000636.jpg     
  inflating: Images/00000637.jpg     
  inflating: Images/00000638.jpg     
  inflating: Images/00000639.jpg     
  inflating: Images/00000640.jpg     
  inflating: Images/00000641.jpg     
  inflating: Images/00000642.jpg     
  inflating: Images/00000643.jpg     
  inflating: Images/00000644.jpg     
  inflating: Images/00000645.jpg     
  inflating: Images/00000646.jpg     
  inflating: Images/00000647.jpg     
  inflating: Images/00000648.jpg     
  inflating: Images/00000649.jpg     
  inflating: Images/00000650.jpg     
  inflating: Images/00000651.jpg     
  inflating: Images/00000652.jpg     
  inflating: Images/00000653.jpg     
  inflating: Images/00000654.jpg     

In [2]:
import tensorflow as tf
import os
import glob as glob
import matplotlib.pyplot as plt
import cv2
import requests
import random
import shutil
import subprocess

In [3]:
# Defining the number of epochs to train.
TRAIN = True
EPOCHS = 5

In [50]:
# Splitting data into train, valid, test
import os, random, shutil

BASE_DIR = "/content/Images"

IMG_DIR = BASE_DIR

TRAIN_DIR = f"{BASE_DIR}/train"
VALID_DIR = f"{BASE_DIR}/valid"
TEST_DIR  = f"{BASE_DIR}/test"

for d in [TRAIN_DIR, VALID_DIR, TEST_DIR]:
    os.makedirs(d, exist_ok=True)

images = [f for f in os.listdir(IMG_DIR) if f.lower().endswith(".jpg")]
random.shuffle(images)

total = len(images)
train_end = int(0.7 * total)
valid_end = int(0.9 * total)

splits = {
    TRAIN_DIR: images[:train_end],
    VALID_DIR: images[train_end:valid_end],
    TEST_DIR:  images[valid_end:]
}

for dest, files in splits.items():
    for img in files:
        shutil.copy(
            os.path.join(IMG_DIR, img),
            os.path.join(dest, img)
        )

for k, v in splits.items():
    print(f"{k}: {len(v)}")



/content/Images/train: 3938
/content/Images/valid: 1125
/content/Images/test: 563


In [5]:
!pip install -q ultralytics
from ultralytics import YOLO
import ultralytics
ultralytics.checks()


Ultralytics 8.4.5 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Setup complete ✅ (12 CPUs, 83.5 GB RAM, 39.1/112.6 GB disk)


In [6]:
# Creating a unique run name (compatible with Ultralytics/YOLOv8)
import os

def set_res_dir(project_dir="runs/train", prefix="results"):
    os.makedirs(project_dir, exist_ok=True)
    existing = [d for d in os.listdir(project_dir) if os.path.isdir(os.path.join(project_dir, d))]
    # count only those that start with prefix_
    n = sum(1 for d in existing if d.startswith(prefix + "_"))
    return f"{prefix}_{n+1}"


In [7]:
import pandas as pd

df = pd.read_csv("labels.csv")
print(df.columns.tolist())
df.head()


['00000000', 'pickup_truck', '213', '34', '255', '50']


,00000000,pickup_truck,213,34,255,50
0,0,car,194,78,273,122
1,0,car,155,27,183,35
2,0,articulated_truck,43,25,109,55
3,0,car,106,32,124,45
4,1,bus,205,155,568,314


In [8]:
from pathlib import Path

OUT_ROOT = Path("/content/dataset_yolo")
for p in [
    OUT_ROOT/"images/train", OUT_ROOT/"images/val",
    OUT_ROOT/"labels/train", OUT_ROOT/"labels/val"
]:
    p.mkdir(parents=True, exist_ok=True)

print("✅ Created:", OUT_ROOT)
!find /content/dataset_yolo -maxdepth 2 -type d -print


✅ Created: /content/dataset_yolo
/content/dataset_yolo
/content/dataset_yolo/images
/content/dataset_yolo/images/train
/content/dataset_yolo/images/val
/content/dataset_yolo/labels
/content/dataset_yolo/labels/train
/content/dataset_yolo/labels/val


In [9]:
import pandas as pd
df = pd.read_csv("/content/labels.csv")
print(df.columns.tolist())
df.head()


['00000000', 'pickup_truck', '213', '34', '255', '50']


,00000000,pickup_truck,213,34,255,50
0,0,car,194,78,273,122
1,0,car,155,27,183,35
2,0,articulated_truck,43,25,109,55
3,0,car,106,32,124,45
4,1,bus,205,155,568,314


In [10]:
import pandas as pd

df = pd.read_csv("/content/labels.csv", header=None)
df.columns = ["img_id", "class_name", "xmin", "ymin", "xmax", "ymax"]
print(df.head())
print("Unique classes:", df["class_name"].unique()[:20], " ... total:", df["class_name"].nunique())


   img_id         class_name  xmin  ymin  xmax  ymax
0       0       pickup_truck   213    34   255    50
1       0                car   194    78   273   122
2       0                car   155    27   183    35
3       0  articulated_truck    43    25   109    55
4       0                car   106    32   124    45
Unique classes: ['pickup_truck' 'car' 'articulated_truck' 'bus' 'motorized_vehicle' 'work_van' 'single_unit_truck' 'pedestrian' 'bicycle' 'non-motorized_vehicle' 'motorcycle']  ... total: 11


In [11]:
import os, shutil
from pathlib import Path
from sklearn.model_selection import train_test_split
import cv2
import pandas as pd

IMAGES_DIR = Path("/content/Images")
OUT_ROOT   = Path("/content/dataset_yolo")

# Create YOLO folder structure
for p in [
    OUT_ROOT/"images/train", OUT_ROOT/"images/val",
    OUT_ROOT/"labels/train", OUT_ROOT/"labels/val"
]:
    p.mkdir(parents=True, exist_ok=True)

# Get image files
img_files = sorted([p for p in IMAGES_DIR.iterdir() if p.suffix.lower() in [".jpg", ".jpeg", ".png"]])
assert len(img_files) > 0, "No images found in /content/Images"

# Determine padding width from filenames like 00000000.jpg
sample_stem = img_files[0].stem
pad_width = len(sample_stem)
print("Images found:", len(img_files))
print("First 5 images:", [p.name for p in img_files[:5]])
print("Detected filename stem width:", pad_width)

# Build a lookup of available stems -> full filename
stem_to_file = {p.stem: p.name for p in img_files}
available_stems = set(stem_to_file.keys())

# Load CSV properly (no header)
df = pd.read_csv("/content/labels.csv", header=None)
df.columns = ["img_id", "class_name", "xmin", "ymin", "xmax", "ymax"]

# Ensure img_id is int, then create stem with zero padding
df["img_id"] = df["img_id"].astype(int)
df["stem"] = df["img_id"].apply(lambda x: str(x).zfill(pad_width))

# Keep only rows that have a matching image file in /content/Images
before = len(df)
df = df[df["stem"].isin(available_stems)].copy()
after = len(df)
print(f"Filtered label rows to existing images: {before} -> {after}")

assert after > 0, "❌ After filtering, no CSV rows matched your available images."

# Map stem -> filename
df["filename"] = df["stem"].map(stem_to_file)

# Class mapping
classes = sorted(df["class_name"].unique().tolist())
class_to_id = {c:i for i,c in enumerate(classes)}
print("Class mapping (first 10):", list(class_to_id.items())[:10])
print("Total classes:", len(classes))

# Train/val split by image stem (not id)
unique_stems = sorted(df["stem"].unique().tolist())
train_stems, val_stems = train_test_split(unique_stems, test_size=0.2, random_state=42)
train_stems = set(train_stems)
val_stems   = set(val_stems)

def yolo_line(xmin, ymin, xmax, ymax, img_w, img_h, cls_id):
    x_c = ((xmin + xmax) / 2.0) / img_w
    y_c = ((ymin + ymax) / 2.0) / img_h
    w   = (xmax - xmin) / img_w
    h   = (ymax - ymin) / img_h
    return f"{cls_id} {x_c:.6f} {y_c:.6f} {w:.6f} {h:.6f}"

def process_split(stems_set, split_name):
    img_out = OUT_ROOT / f"images/{split_name}"
    lbl_out = OUT_ROOT / f"labels/{split_name}"

    for stem in stems_set:
        fname = stem_to_file[stem]
        src_img = IMAGES_DIR / fname

        shutil.copy2(src_img, img_out / fname)

        img = cv2.imread(str(src_img))
        if img is None:
            continue
        h_img, w_img = img.shape[:2]

        sub = df[df["stem"] == stem]

        lines = []
        for _, r in sub.iterrows():
            cls_id = class_to_id[r["class_name"]]
            xmin, ymin, xmax, ymax = map(float, [r["xmin"], r["ymin"], r["xmax"], r["ymax"]])

            # clip boxes
            xmin = max(0, min(xmin, w_img - 1))
            xmax = max(0, min(xmax, w_img - 1))
            ymin = max(0, min(ymin, h_img - 1))
            ymax = max(0, min(ymax, h_img - 1))

            if xmax <= xmin or ymax <= ymin:
                continue

            lines.append(yolo_line(xmin, ymin, xmax, ymax, w_img, h_img, cls_id))

        (lbl_out / f"{stem}.txt").write_text("\n".join(lines))

process_split(train_stems, "train")
process_split(val_stems, "val")

print("✅ Created dataset at:", OUT_ROOT)
print("Train images:", len(list((OUT_ROOT/'images/train').glob('*.*'))))
print("Val images:", len(list((OUT_ROOT/'images/val').glob('*.*'))))
print("Train labels:", len(list((OUT_ROOT/'labels/train').glob('*.txt'))))
print("Val labels:", len(list((OUT_ROOT/'labels/val').glob('*.txt'))))


Images found: 5626
First 5 images: ['00000000.jpg', '00000001.jpg', '00000002.jpg', '00000003.jpg', '00000004.jpg']
Detected filename stem width: 8
Filtered label rows to existing images: 351549 -> 17967
Class mapping (first 10): [('articulated_truck', 0), ('bicycle', 1), ('bus', 2), ('car', 3), ('motorcycle', 4), ('motorized_vehicle', 5), ('non-motorized_vehicle', 6), ('pedestrian', 7), ('pickup_truck', 8), ('single_unit_truck', 9)]
Total classes: 11
✅ Created dataset at: /content/dataset_yolo
Train images: 4500
Val images: 1126
Train labels: 4500
Val labels: 1126


In [12]:
import yaml
from pathlib import Path

OUT_ROOT = Path("/content/dataset_yolo")

# Recompute classes from the filtered CSV if needed
import pandas as pd
df2 = pd.read_csv("/content/labels.csv", header=None)
df2.columns = ["img_id", "class_name", "xmin", "ymin", "xmax", "ymax"]
# Keep same class list as before, but it’s okay if subset has fewer
classes = sorted(df["class_name"].unique().tolist())

data = {
    "path": str(OUT_ROOT),
    "train": "images/train",
    "val": "images/val",
    "nc": len(classes),
    "names": classes
}

yaml_path = Path("/content/data.yaml")
with open(yaml_path, "w") as f:
    yaml.safe_dump(data, f, sort_keys=False)

print("✅ Wrote:", yaml_path)
print(yaml_path.read_text())


✅ Wrote: /content/data.yaml
path: /content/dataset_yolo
train: images/train
val: images/val
nc: 11
names:
- articulated_truck
- bicycle
- bus
- car
- motorcycle
- motorized_vehicle
- non-motorized_vehicle
- pedestrian
- pickup_truck
- single_unit_truck
- work_van



In [13]:
# Show common YOLOv8 training artifacts (labels + sample predictions)
import os, glob
import matplotlib.pyplot as plt
import cv2

def show_train_artifacts(res_dir, project='runs/train'):
    run_path = os.path.join(project, res_dir)
    print('Run path:', run_path)
    if not os.path.isdir(run_path):
        raise FileNotFoundError(run_path)

    # Common outputs: results.png, confusion_matrix.png, PR_curve.png, val_batch*_pred.jpg
    candidates = []
    candidates += glob.glob(os.path.join(run_path, '*.png'))
    candidates += glob.glob(os.path.join(run_path, 'val_batch*_pred.jpg'))
    candidates += glob.glob(os.path.join(run_path, 'train_batch*.jpg'))

    print(f'Found {len(candidates)} artifact images')
    for p in sorted(candidates)[:20]:
        img = cv2.imread(p)
        if img is None:
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(12, 8))
        plt.imshow(img)
        plt.axis('off')
        plt.title(os.path.basename(p))
        plt.show()


In [14]:
from ultralytics import YOLO
import os
import glob

# Helper: run YOLOv8 inference and return the save_dir (where annotated images are written).
def inference_yolov8(weights_path, source, conf=0.15, imgsz=640, project='runs/detect', name='infer', exist_ok=True):
    if not os.path.exists(weights_path):
        raise FileNotFoundError(f"Weights not found: {weights_path}")

    model = YOLO(weights_path)
    results = model.predict(
        source=source,
        conf=conf,
        imgsz=imgsz,
        save=True,
        project=project,
        name=name,
        exist_ok=exist_ok
    )
    # results is a list; save_dir is the same for all entries
    return str(results[0].save_dir)

# Example usage (uncomment and update paths):
# save_dir = inference_yolov8('/content/runs/train/results_1/weights/best.pt', '/content/dataset/test/images', conf=0.15)
# print('Saved predictions to:', save_dir)


In [15]:
# Step 10
def visualize(infer_dir):
    infer_path = os.path.join("runs", "train", infer_dir)

    if not os.path.exists(infer_path):
        print("Inference directory not found:", infer_path)
        return

    infer_images = sorted(glob.glob(os.path.join(infer_path, "*.jpg")))

    print(f"Found {len(infer_images)} inference images")

    for pred_image in infer_images:
        image = cv2.imread(pred_image)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        plt.figure(figsize=(19, 16))
        plt.imshow(image)
        plt.title(os.path.basename(pred_image))
        plt.axis("off")
        plt.show()

In [16]:
import pandas as pd

CSV_PATH = "/content/labels.csv"
df = pd.read_csv(CSV_PATH, header=None, names=["image_id","label","xmin","ymin","xmax","ymax"])
print(df.head())
print(df.columns.tolist())


   image_id              label  xmin  ymin  xmax  ymax
0         0       pickup_truck   213    34   255    50
1         0                car   194    78   273   122
2         0                car   155    27   183    35
3         0  articulated_truck    43    25   109    55
4         0                car   106    32   124    45
['image_id', 'label', 'xmin', 'ymin', 'xmax', 'ymax']


In [17]:
import os
from PIL import Image

IMAGES_DIR = "/content/Images"        # folder containing ALL images (or your train images)
OUT_LABELS_DIR = "/content/labels_yolo"
os.makedirs(OUT_LABELS_DIR, exist_ok=True)

# class mapping
classes = sorted(df["label"].unique().tolist())
class_to_id = {c:i for i,c in enumerate(classes)}
print("Class map:", class_to_id)

def clamp(v, lo, hi):
    return max(lo, min(hi, v))

missing = 0
written = 0

for img_id, g in df.groupby("image_id"):
    # convert 0 -> "00000000.jpg"
    fname = f"{int(img_id):08d}.jpg"
    img_path = os.path.join(IMAGES_DIR, fname)

    if not os.path.exists(img_path):
        # try png too
        fname_png = f"{int(img_id):08d}.png"
        img_path_png = os.path.join(IMAGES_DIR, fname_png)
        if os.path.exists(img_path_png):
            fname, img_path = fname_png, img_path_png
        else:
            missing += 1
            continue

    w, h = Image.open(img_path).size

    lines = []
    for _, row in g.iterrows():
        cls = class_to_id[row["label"]]
        xmin, ymin, xmax, ymax = map(float, (row["xmin"], row["ymin"], row["xmax"], row["ymax"]))

        xmin = clamp(xmin, 0, w-1)
        xmax = clamp(xmax, 0, w-1)
        ymin = clamp(ymin, 0, h-1)
        ymax = clamp(ymax, 0, h-1)

        bw = (xmax - xmin) / w
        bh = (ymax - ymin) / h
        if bw <= 0 or bh <= 0:
            continue

        x_center = ((xmin + xmax) / 2) / w
        y_center = ((ymin + ymax) / 2) / h
        lines.append(f"{cls} {x_center:.6f} {y_center:.6f} {bw:.6f} {bh:.6f}")

    out_txt = os.path.join(OUT_LABELS_DIR, os.path.splitext(fname)[0] + ".txt")
    with open(out_txt, "w") as f:
        f.write("\n".join(lines))

    written += 1

print("Wrote label files for images:", written)
print("Missing images (no file found):", missing)
print("Example label files:", os.listdir(OUT_LABELS_DIR)[:5])


Class map: {'articulated_truck': 0, 'bicycle': 1, 'bus': 2, 'car': 3, 'motorcycle': 4, 'motorized_vehicle': 5, 'non-motorized_vehicle': 6, 'pedestrian': 7, 'pickup_truck': 8, 'single_unit_truck': 9, 'work_van': 10}
Wrote label files for images: 5626
Missing images (no file found): 104374
Example label files: ['00002823.txt', '00000498.txt', '00005488.txt', '00000562.txt', '00000961.txt']


In [18]:
%cd /content
!mkdir -p dataset/train/images dataset/train/labels dataset/valid/images dataset/valid/labels dataset/test/images dataset/test/labels

# copy images into YOLO structure
!cp -n /content/Images/train/* dataset/train/images/ 2>/dev/null || true
!cp -n /content/Images/valid/* dataset/valid/images/ 2>/dev/null || true
!cp -n /content/Images/test/*  dataset/test/images/ 2>/dev/null || true


/content


In [19]:
import os, shutil, glob

YOLO_LABELS = "/content/labels_yolo"
splits = ["train","valid","test"]

for split in splits:
    img_dir = f"/content/dataset/{split}/images"
    lbl_dir = f"/content/dataset/{split}/labels"
    os.makedirs(lbl_dir, exist_ok=True)

    for img_path in glob.glob(os.path.join(img_dir, "*.jpg")) + glob.glob(os.path.join(img_dir, "*.png")):
        base = os.path.splitext(os.path.basename(img_path))[0]
        src_lbl = os.path.join(YOLO_LABELS, base + ".txt")
        if os.path.exists(src_lbl):
            shutil.copy(src_lbl, os.path.join(lbl_dir, base + ".txt"))

print("Done copying labels into dataset splits.")


Done copying labels into dataset splits.


In [20]:
%%writefile /content/data.yaml
train: /content/dataset/train/images
val: /content/dataset/valid/images

nc: 2
names: ['articulated_truck', 'car']


Overwriting /content/data.yaml


In [21]:
# Remapping all label files in train/valid/test:
import glob

# map old_id -> new_id
remap = {2: 0, 3: 1}

label_dirs = [
    "/content/dataset/train/labels",
    "/content/dataset/valid/labels",
    "/content/dataset/test/labels",
]

changed = 0
skipped = 0

for d in label_dirs:
    for path in glob.glob(d + "/*.txt"):
        new_lines = []
        ok = True
        for line in open(path, "r"):
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            cid = int(float(parts[0]))
            if cid not in remap:
                ok = False
                break
            parts[0] = str(remap[cid])
            new_lines.append(" ".join(parts))

        if not ok:
            skipped += 1
            continue

        with open(path, "w") as f:
            f.write("\n".join(new_lines))
        changed += 1

print("Rewrote label files:", changed)
print("Skipped (had unexpected class ids):", skipped)


Rewrote label files: 2463
Skipped (had unexpected class ids): 3163


In [22]:
import glob

ids = set()
for p in glob.glob("/content/dataset/train/labels/*.txt"):
    for line in open(p, "r"):
        parts = line.strip().split()
        if len(parts) == 5:
            ids.add(int(float(parts[0])))

ids = sorted(ids)
print("Unique class IDs found:", ids)
print("Min:", min(ids), "Max:", max(ids), "Count:", len(ids))

Unique class IDs found: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Min: 0 Max: 10 Count: 11


In [23]:
sorted(df["label"].unique().tolist())[:50], len(df["label"].unique())


(['articulated_truck',
  'bicycle',
  'bus',
  'car',
  'motorcycle',
  'motorized_vehicle',
  'non-motorized_vehicle',
  'pedestrian',
  'pickup_truck',
  'single_unit_truck',
  'work_van'],
 11)

In [24]:
K = max(ids)
names = [str(i) for i in range(K+1)]
print("nc =", K+1)
print("names =", names[:10], "...")


nc = 11
names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] ...


In [25]:
yaml = "train: /content/dataset/train/images\nval: /content/dataset/valid/images\n\n"
yaml += f"nc: {K+1}\n"
yaml += "names: [" + ", ".join([f"'{n}'" for n in names]) + "]\n"

open("/content/data.yaml", "w").write(yaml)
print(open("/content/data.yaml").read()[:300])


train: /content/dataset/train/images
val: /content/dataset/valid/images

nc: 11
names: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10']



In [26]:
# (YOLOv8) No need to cd into a cloned repo.


In [27]:
!ls


dataset  dataset_yolo  data.yaml  Images  Images.zip  labels.csv  labels_yolo


In [28]:
# YOLOv8: install Ultralytics (if you haven't already)
# If you already ran the earlier `pip install ultralytics` cell, you can skip this.
!pip -q install ultralytics


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [29]:
# (YOLOv8) Your weights will be saved under runs/train/<run_name>/weights/


In [30]:
# Alternate training cell (YOLOv8) - larger image size example
model = YOLO('yolov8n.pt')
results = model.train(
    data='/content/data.yaml',
    imgsz=620,
    epochs=10,
    batch=8,
    project='runs/train',
    name='results_1',
    exist_ok=True
)
print("Saved to:", results.save_dir)


Ultralytics 8.4.5 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=620, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=results_1, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=

In [31]:
# Apply fixed code

In [32]:
yaml_text = """train: /content/dataset/train/images
val: /content/dataset/valid/images
test: /content/dataset/test/images

nc: 11
names:
  - articulated_truck
  - bicycle
  - bus
  - car
  - motorcycle
  - motorized_vehicle
  - non-motorized_vehicle
  - pedestrian
  - pickup_truck
  - single_unit_truck
  - work_van
"""
open("/content/data.yaml","w").write(yaml_text)
print("✅ data.yaml restored")


✅ data.yaml restored


In [33]:
import yaml

with open("/content/data.yaml","r") as f:
    names = yaml.safe_load(f)["names"]

line = "3 0.0694444 0.0697917 0.0555556 0.0395833 0.3882"
parts = line.split()
cls_id = int(parts[0])
conf = float(parts[5])

print("Class name:", names[cls_id])
print("Confidence:", conf)


Class name: car
Confidence: 0.3882


In [34]:
!cat /content/data.yaml


train: /content/dataset/train/images
val: /content/dataset/valid/images
test: /content/dataset/test/images

nc: 11
names:
  - articulated_truck
  - bicycle
  - bus
  - car
  - motorcycle
  - motorized_vehicle
  - non-motorized_vehicle
  - pedestrian
  - pickup_truck
  - single_unit_truck
  - work_van


In [ ]:
import os, glob
from IPython.display import Image, display

# Display a few YOLOv8 annotated outputs (from the previous predict() cell).
# If you changed the predict() name/project, update PRED_OUT below.
PRED_OUT = '/content/runs/detect/sample_infer'

outs = sorted(glob.glob(os.path.join(PRED_OUT, '*.jpg')) + glob.glob(os.path.join(PRED_OUT, '*.png')))
print('Annotated outputs:', len(outs))

# Show up to first 5 images
for p in outs[:5]:
    display(Image(filename=p))


In [37]:
# Inferencing sample images

In [46]:
!cp /content/dataset/test/images/*.jpg /content/samples/ 2>/dev/null
!cp /content/dataset/test/images/*.png /content/samples/ 2>/dev/null


In [47]:
from ultralytics import YOLO

%cd /content
!mkdir -p samples

weights_path = '/content/runs/train/results_1/weights/best.pt'  # update if needed
model = YOLO(weights_path)

results = model.predict(
    source='/content/samples',
    conf=0.15,
    save=True,
    project='runs/detect',
    name='sample_infer',
    exist_ok=True
)
print("Saved predictions to:", results[0].save_dir)


/content

image 1/563 /content/samples/00000006.jpg: 448x640 1 1, 69.1ms
image 2/563 /content/samples/00000007.jpg: 448x640 1 1, 7.7ms
image 3/563 /content/samples/00000015.jpg: 448x640 2 1s, 4 3s, 1 10, 7.1ms
image 4/563 /content/samples/00000017.jpg: 448x640 1 1, 7.0ms
image 5/563 /content/samples/00000027.jpg: 448x640 1 1, 1 3, 1 8, 6.8ms
image 6/563 /content/samples/00000049.jpg: 448x640 2 1s, 2 3s, 2 5s, 6.8ms
image 7/563 /content/samples/00000057.jpg: 448x640 2 1s, 3 3s, 1 8, 6.7ms
image 8/563 /content/samples/00000059.jpg: 448x640 3 1s, 1 3, 1 8, 6.7ms
image 9/563 /content/samples/00000063.jpg: 448x640 2 1s, 1 3, 6.8ms
image 10/563 /content/samples/00000085.jpg: 448x640 2 1s, 26 3s, 1 8, 7.4ms
image 11/563 /content/samples/00000090.jpg: 448x640 1 1, 1 3, 1 10, 6.8ms
image 12/563 /content/samples/00000105.jpg: 448x640 2 1s, 4 3s, 1 8, 6.8ms
image 13/563 /content/samples/00000108.jpg: 448x640 1 1, 6.8ms
image 14/563 /content/samples/00000114.jpg: 448x640 1 0, 1 2, 6.8ms
image 15/5

In [48]:
import os, glob

DATASET_DIR = "/content/dataset"   # change if yours differs
splits = ["train", "valid","test"]        # add "test" if you have it

def check_split(split):
    img_dir = os.path.join(DATASET_DIR, split, "images")
    lbl_dir = os.path.join(DATASET_DIR, split, "labels")
    imgs = sorted(glob.glob(os.path.join(img_dir, "*.*")))
    lbls = sorted(glob.glob(os.path.join(lbl_dir, "*.txt")))

    img_stems = {os.path.splitext(os.path.basename(p))[0] for p in imgs}
    lbl_stems = {os.path.splitext(os.path.basename(p))[0] for p in lbls}

    missing_lbl = sorted(list(img_stems - lbl_stems))[:20]
    extra_lbl = sorted(list(lbl_stems - img_stems))[:20]

    print(f"\n=== {split.upper()} ===")
    print("Images:", len(imgs))
    print("Labels:", len(lbls))
    print("Missing label files for images (first 20):", missing_lbl)
    print("Label files with no matching image (first 20):", extra_lbl)

    # Content checks
    empty = 0
    bad_lines = 0
    bad_range = 0
    class_ids = set()

    for lp in lbls[:5000]:  # scan up to 5000 label files
        txt = open(lp, "r").read().strip()
        if not txt:
            empty += 1
            continue
        for line in txt.splitlines():
            parts = line.split()
            if len(parts) != 5:
                bad_lines += 1
                continue
            try:
                c = int(float(parts[0]))
                x, y, w, h = map(float, parts[1:])
                class_ids.add(c)
            except:
                bad_lines += 1
                continue

            # normalized range checks
            if not (0 <= x <= 1 and 0 <= y <= 1 and 0 < w <= 1 and 0 < h <= 1):
                bad_range += 1

    print("Empty label files:", empty)
    print("Bad-format lines:", bad_lines)
    print("Out-of-range (not normalized) boxes:", bad_range)
    if class_ids:
        print("Class IDs found (min/max/count):", min(class_ids), max(class_ids), len(class_ids))
    else:
        print("Class IDs found: NONE (labels may be empty/unreadable)")

for s in splits:
    check_split(s)



=== TRAIN ===
Images: 3938
Labels: 3938
Missing label files for images (first 20): []
Label files with no matching image (first 20): []
Empty label files: 0
Bad-format lines: 0
Out-of-range (not normalized) boxes: 0
Class IDs found (min/max/count): 0 10 11

=== VALID ===
Images: 1125
Labels: 1125
Missing label files for images (first 20): []
Label files with no matching image (first 20): []
Empty label files: 0
Bad-format lines: 0
Out-of-range (not normalized) boxes: 0
Class IDs found (min/max/count): 0 10 11

=== TEST ===
Images: 563
Labels: 563
Missing label files for images (first 20): []
Label files with no matching image (first 20): []
Empty label files: 0
Bad-format lines: 0
Out-of-range (not normalized) boxes: 0
Class IDs found (min/max/count): 0 10 11


In [53]:
import os, glob, yaml

DATA_YAML = "/content/data.yaml"  # change if needed
d = yaml.safe_load(open(DATA_YAML))

for k in ["train", "val", "test"]:
    p = d[k]
    imgs = glob.glob(os.path.join(p, "*.*"))
    print(k, "->", p, "| exists:", os.path.exists(p), "| images:", len(imgs))


train -> /content/dataset/train/images | exists: True | images: 3938
val -> /content/dataset/valid/images | exists: True | images: 1125
test -> /content/dataset/test/images | exists: True | images: 563
